## MLP PURO

In [1]:
# mlp_regressao_baseline.py
# ============================================================
# Regressão de "Precipitação dia" com MLP (PyTorch) - BASELINE
# Com SEED e LR_BASELINE para reprodutibilidade.
# Sem poda / quantização / qualquer otimização.
# ============================================================

import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# ----------------------- Configuração -----------------------
CSV_PATH = "bd_EstacaoVargemFria_e_Pesca.csv"

EPOCHS_BASELINE = 100
LR_BASELINE = 1e-3
BATCH_SIZE = 64
SEED = 42

DEVICE = torch.device("cpu")  # forçado CPU

# ----------------------- Utilidades -----------------------
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)

def mae(y_true, y_pred):
    return float(np.mean(np.abs(y_pred - y_true)))

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_pred - y_true) ** 2)))

def r2_score_np(y_true, y_pred):
    ss_res = float(np.sum((y_true - y_pred) ** 2))
    ss_tot = float(np.sum((y_true - np.mean(y_true)) ** 2))
    return float(1.0 - ss_res / ss_tot) if ss_tot > 0 else float("nan")

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ----------------------- Dataset -----------------------
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))              # (N, F)
        self.y = torch.from_numpy(y.astype(np.float32).reshape(-1))  # (N,)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ----------------------- Modelo -----------------------
class MLPRegressor(nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(1)  # (N,)

# ----------------------- Treino / Avaliação -----------------------
def train_model(model, loader, epochs, lr):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        for xb, yb in loader:
            preds = model(xb)            # CPU
            loss = criterion(preds, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * xb.size(0)
        epoch_loss /= len(loader.dataset)
        if epoch % 5 == 0 or epoch == 1 or epoch == epochs:
            print(f"Epoch {epoch:02d}/{epochs}  - Train MSE: {epoch_loss:.6f}")

def evaluate_model(model, loader, label="Avaliação"):
    model.eval()
    y_true, y_pred = [], []
    t0 = time.time()
    with torch.no_grad():
        for xb, yb in loader:
            preds = model(xb).cpu().numpy()
            y_pred.append(preds)
            y_true.append(yb.cpu().numpy())
    t1 = time.time()

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    _mae = mae(y_true, y_pred)
    _rmse = rmse(y_true, y_pred)
    _r2 = r2_score_np(y_true, y_pred)
    total_time = t1 - t0
    time_per_sample_ms = (total_time / len(loader.dataset)) * 1000.0

    print(f"\n📊 {label}:")
    print(f"MAE:   {_mae:.4f}")
    print(f"RMSE:  {_rmse:.4f}")
    print(f"R²:    {_r2:.4f}")
    print(f"Tempo total de inferência: {total_time:.6f} s")
    print(f"Tempo médio por amostra:   {time_per_sample_ms:.4f} ms")

    return _mae, _rmse, _r2, time_per_sample_ms

# ----------------------- Main -----------------------
def main():
    set_seed(SEED)

    # ============================================================
    # PRÉ-PROCESSAMENTO (idêntico ao da CNN baseline)
    # ============================================================
    df = pd.read_csv(CSV_PATH)
    df = df[df['Nome'].str.strip() == 'Estação Pesca - UFRPE']
    df['Data estação'] = pd.to_datetime(df['Data estação'], errors='coerce')
    df = df.sort_values('Data estação')
    df = df.interpolate(method='linear', limit_direction='forward')
    df['Precipitação anterior'] = df['Precipitação dia'].shift(1)
    df = df.dropna()

    colunas_features = ['Temperatura', 'Umidade', 'Velocidade Vento', 'Rajada Vento', 'Precipitação anterior']
    coluna_saida = 'Precipitação dia'
    df = df[colunas_features + [coluna_saida]]

    X = df[colunas_features].values
    y = df[coluna_saida].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, random_state=SEED
    )

    train_ds = TabularDataset(X_train, y_train)
    test_ds  = TabularDataset(X_test,  y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

    # ============================================================
    # BASELINE: treino e avaliação
    # ============================================================
    model = MLPRegressor(in_features=X_train.shape[1]).to(DEVICE)
    print(f"Parâmetros treináveis: {count_params(model):,}")

    print("\n=== Treinando baseline (sem otimizações) ===")
    train_model(model, train_loader, epochs=EPOCHS_BASELINE, lr=LR_BASELINE)
    evaluate_model(model, test_loader, label="Baseline MLP (sem otimizações)")

if __name__ == "__main__":
    main()


Parâmetros treináveis: 2,497

=== Treinando baseline (sem otimizações) ===
Epoch 01/100  - Train MSE: 6.841244
Epoch 05/100  - Train MSE: 0.259598
Epoch 10/100  - Train MSE: 0.256308
Epoch 15/100  - Train MSE: 0.254737
Epoch 20/100  - Train MSE: 0.252864
Epoch 25/100  - Train MSE: 0.250617
Epoch 30/100  - Train MSE: 0.255069
Epoch 35/100  - Train MSE: 0.252574
Epoch 40/100  - Train MSE: 0.251391
Epoch 45/100  - Train MSE: 0.250561
Epoch 50/100  - Train MSE: 0.249480
Epoch 55/100  - Train MSE: 0.252669
Epoch 60/100  - Train MSE: 0.248716
Epoch 65/100  - Train MSE: 0.250889
Epoch 70/100  - Train MSE: 0.251528
Epoch 75/100  - Train MSE: 0.249430
Epoch 80/100  - Train MSE: 0.241440
Epoch 85/100  - Train MSE: 0.251638
Epoch 90/100  - Train MSE: 0.244204
Epoch 95/100  - Train MSE: 0.249912
Epoch 100/100  - Train MSE: 0.246966

📊 Baseline MLP (sem otimizações):
MAE:   0.0887
RMSE:  1.0670
R²:    0.9327
Tempo total de inferência: 0.100602 s
Tempo médio por amostra:   0.0167 ms


## Puro com energia